In [1]:
import scipy.io
import pandas as pd

mat = scipy.io.loadmat('Xtrain.mat')

In [2]:
df = pd.DataFrame(mat['Xtrain'])
original_df = pd.DataFrame(mat['Xtrain'])

print (df)

       0
0     86
1    141
2     95
3     41
4     22
..   ...
995   61
996   20
997   12
998   13
999   23

[1000 rows x 1 columns]


In [3]:
# Scale the data (sklearn minMax scaler)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
scaler.data_max_
df = scaler.transform(df)
print(df)

[[0.33201581]
 [0.54940711]
 [0.36758893]
 [0.1541502 ]
 [0.07905138]
 [0.07509881]
 [0.11857708]
 [0.27667984]
 [0.53754941]
 [0.43083004]
 [0.18181818]
 [0.08300395]
 [0.06719368]
 [0.09881423]
 [0.22529644]
 [0.50197628]
 [0.50197628]
 [0.22134387]
 [0.09881423]
 [0.06719368]
 [0.08695652]
 [0.17391304]
 [0.43478261]
 [0.56126482]
 [0.28063241]
 [0.11067194]
 [0.07114625]
 [0.06719368]
 [0.13833992]
 [0.35573123]
 [0.59288538]
 [0.35968379]
 [0.13438735]
 [0.07114625]
 [0.06324111]
 [0.10671937]
 [0.27272727]
 [0.56916996]
 [0.45454545]
 [0.17391304]
 [0.08300395]
 [0.06324111]
 [0.07905138]
 [0.19762846]
 [0.49802372]
 [0.55335968]
 [0.23715415]
 [0.09486166]
 [0.05928854]
 [0.06719368]
 [0.13833992]
 [0.38735178]
 [0.61660079]
 [0.33201581]
 [0.11857708]
 [0.05928854]
 [0.05928854]
 [0.09881423]
 [0.27667984]
 [0.60079051]
 [0.45849802]
 [0.16205534]
 [0.07114625]
 [0.0513834 ]
 [0.07509881]
 [0.17786561]
 [0.49802372]
 [0.58498024]
 [0.24110672]
 [0.08695652]
 [0.05533597]
 [0.05

In [4]:
# Create array with windows. Note: use Numpy X = X.reshape

window_size = 3 #parameter needs tuning!
X_features = list()
y_features = list()

index = 0
while (index + window_size) < len(df):
    windowlist = list()
    for i in range(window_size):
        value = (df[index+i][0])
        windowlist.append(value)
#    print(windowlist)

    X_features.append(windowlist)
    y = df[index+window_size]
    y_features.append(y[0])
    index = index + 1
    

print(X_features)
print(y_features)

[[0.3320158102766798, 0.549407114624506, 0.3675889328063241], [0.549407114624506, 0.3675889328063241, 0.15415019762845847], [0.3675889328063241, 0.15415019762845847, 0.07905138339920949], [0.15415019762845847, 0.07905138339920949, 0.07509881422924901], [0.07905138339920949, 0.07509881422924901, 0.11857707509881422], [0.07509881422924901, 0.11857707509881422, 0.27667984189723316], [0.11857707509881422, 0.27667984189723316, 0.5375494071146245], [0.27667984189723316, 0.5375494071146245, 0.43083003952569165], [0.5375494071146245, 0.43083003952569165, 0.1818181818181818], [0.43083003952569165, 0.1818181818181818, 0.08300395256916995], [0.1818181818181818, 0.08300395256916995, 0.06719367588932806], [0.08300395256916995, 0.06719367588932806, 0.09881422924901186], [0.06719367588932806, 0.09881422924901186, 0.225296442687747], [0.09881422924901186, 0.225296442687747, 0.5019762845849802], [0.225296442687747, 0.5019762845849802, 0.5019762845849802], [0.5019762845849802, 0.5019762845849802, 0.2213

In [5]:
# Create LSTM model (Tensorflow)

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def generate_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

sequence_length = 3#needs tuning!
X, y = generate_sequences(df[:799], sequence_length)
print(X,y)
model = Sequential([
    LSTM(3, input_shape=(sequence_length, 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=10, batch_size=32)

ImportError: Traceback (most recent call last):
  File "c:\Users\Collin\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Een initialisatieroutine van de dynamic link library (DLL-bestand) is mislukt.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
# Recursively predict one-by-one
output = model.predict(df[797:800])
#print(output)
#print(df[800][0])

# rescaling y_pred
output_rescaled = scaler.inverse_transform(output)
y_pred = sum(output_rescaled[0]) / len(output_rescaled[0])
print(y_pred)

# rescaling y_true
gt=df[800][0]
y_true = scaler.inverse_transform([[gt]])
y_true = y_true[0][0]
print(y_true)

### now predicting away the rest!
index = 0
testset_y_pred = list()

initialWindow = df[797:800]
print("---")
for i in range (200):
    if i == 0:
        window = initialWindow
        print("first window: ", window)
    # processing the current window
    output = model.predict(window)
    output_rescaled = scaler.inverse_transform(output)
    y_pred = sum(output_rescaled[0]) / len(output_rescaled[0])
    testset_y_pred.append(y_pred)
    # preparing new window for the next iteration
    window = np.append(window, y_pred)
    window = np.delete(window, 0)
    window = [[(float(window[0]))],[(float(window[1]))],[(float(window[2]))]]
    window = np.array(window)

    print("new window = ", (window))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
47.42359924316406
41.0
---
first window:  [[0.35968379]
 [0.43083004]
 [0.28458498]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
new window =  [[ 0.43083004]
 [ 0.28458498]
 [47.42359924]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
new window =  [[ 0.28458498]
 [47.42359924]
 [48.70161438]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
new window =  [[47.42359924]
 [48.70161438]
 [45.98072815]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
new window =  [[48.70161438]
 [45.98072815]
 [28.20678329]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
new window =  [[45.98072815]
 [28.20678329]
 [28.39792061]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
new window =  [[28.20678329]
 [28.39792061]
 [27.97918129]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
new window =  [[28.39792061]
 [27.97918129]
 [22.90089035]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
new window =  [[27.97918129]
 [22.90089035]
 [22.99591637]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
new window =  [[22.90089035]
 [22.99591637]
 [22.78592

In [ ]:
# Compare with test set output (last 200) and calculate MSE

testset_y_true = df[800:]
for ele in testset_y_true:
    ele = scaler.inverse_transform([ele])
    
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(testset_y_pred, testset_y_true)


NameError: name 'testset_y_pred' is not defined

In [ ]:
print("plain mae: ", mae)
print("transformed mae: ", scaler.transform([[mae]]))

NameError: name 'mae' is not defined

acknowledgements: https://archive.is/gPSyk